# Diagnóstico e Correção - Deploy Vercel AI-Synth 

## Problema Identificado
❌ **Sistema retornando scores idênticos para diferentes gêneros musicais na Vercel**  
❌ **Arquivos de referência não encontrados (404)**  
❌ **Scoring unificado não funcionando em produção**

## Análise Crítica dos Logs
O console mostra que o sistema scoring.js está sendo carregado (`🎯 NOVO SISTEMA CARREGADO - Versão: 2.0.0-equal-weight-v3-FORCED`) mas não está executando corretamente, resultando em fallback para sistema antigo.

**Data:** 26 de agosto de 2025  
**Versão:** AI-Synth v2.0.0 Equal Weight V3 Unificado  
**Deploy:** Vercel Production

## 1. Configuração do Ambiente

Configurando ferramentas necessárias para diagnóstico e correção do deploy na Vercel.

In [ ]:
import os
import json
import subprocess
import requests
from pathlib import Path
import time
from datetime import datetime

# Configurações do projeto
PROJECT_PATH = r"c:\Users\DJ Correa\Desktop\Programação\ai-synth"
VERCEL_URL = "https://ai-synth.vercel.app"
REFS_PATHS = [
    "refs/out/funk_mandela.json",
    "refs/out/trance.json", 
    "refs/out/eletronico.json"
]

print("🔧 CONFIGURAÇÃO DO AMBIENTE")
print(f"📁 Projeto: {PROJECT_PATH}")
print(f"🌐 Vercel URL: {VERCEL_URL}")
print(f"📋 Refs a verificar: {len(REFS_PATHS)} arquivos")

# Verificar se estamos no diretório correto
if os.path.exists(os.path.join(PROJECT_PATH, "package.json")):
    print("✅ Diretório do projeto encontrado")
else:
    print("❌ Diretório do projeto não encontrado")

## 2. Análise dos Logs do Console

Analisando os logs fornecidos para identificar problemas específicos.

In [ ]:
# Análise dos logs críticos do console
console_logs = """
Failed to load resource: the server responded with a status of 404 ()
public/refs/out/funk_mandela.json?v=1756243614789&v=1756243614789:1

[refs] Falha 404 em /public/refs/out/funk_mandela.json?v=1756243614789
[refs] tentando fetch: /refs/out/funk_mandela.json?v=1756243614789&v=1756243614875
[refs] Falha 404 em /refs/out/funk_mandela.json?v=1756243614789

scoring.js?v=1756243635990:1044 🎯 NOVO SISTEMA CARREGADO - Versão: 2.0.0-equal-weight-v3-FORCED
[SCORE_DEBUG] scoring.js carregado: true
[SCORE_DEBUG] computeMixScore disponível: true
[SCORE_DEBUG] ⚠️ qualityOverall inválido, aplicando fallback final
[WEIGHTED_AGGREGATE] Set qualityOverall = 36.55 from 5 sub-scores
"""

print("🔍 ANÁLISE DOS LOGS DO CONSOLE")
print("="*50)

# Identificar problemas principais
problemas_identificados = {
    "refs_404": "❌ Arquivos de referência não encontrados (404)",
    "scoring_loaded": "✅ Scoring.js carregado com sucesso", 
    "scoring_invalid": "❌ Scoring retorna resultado inválido",
    "fallback_triggered": "⚠️ Sistema de fallback ativado"
}

for problema, descricao in problemas_identificados.items():
    print(f"{descricao}")

print("\n🎯 DIAGNÓSTICO PRINCIPAL:")
print("1. Os arquivos de referência não estão sendo encontrados na Vercel")
print("2. O scoring.js unificado carrega mas não executa corretamente")
print("3. Sistema cai no fallback antigo que não diferencia gêneros")

## 3. Verificação de Arquivos de Referência

Verificando a existência e estrutura dos arquivos de referência localmente.

In [ ]:
# Verificar arquivos de referência no projeto local
print("📋 VERIFICAÇÃO DE ARQUIVOS DE REFERÊNCIA")
print("="*50)

refs_status = {}

for ref_path in REFS_PATHS:
    full_path = os.path.join(PROJECT_PATH, ref_path)
    public_path = os.path.join(PROJECT_PATH, "public", ref_path)
    
    local_exists = os.path.exists(full_path)
    public_exists = os.path.exists(public_path)
    
    refs_status[ref_path] = {
        "local": local_exists,
        "public": public_exists,
        "full_path": full_path,
        "public_path": public_path
    }
    
    print(f"\n📄 {ref_path}")
    print(f"   Local (refs/): {'✅' if local_exists else '❌'}")
    print(f"   Public (public/refs/): {'✅' if public_exists else '❌'}")
    
    # Se existir, verificar conteúdo
    if local_exists:
        try:
            with open(full_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                print(f"   Conteúdo: {len(data)} chaves")
                # Verificar chaves importantes
                important_keys = ['lufs_target', 'dr_target', 'stereo_target']
                has_keys = [key for key in important_keys if key in data]
                print(f"   Chaves importantes: {len(has_keys)}/{len(important_keys)}")
        except Exception as e:
            print(f"   ❌ Erro ao ler: {e}")

print(f"\n📊 RESUMO:")
total_local = sum(1 for s in refs_status.values() if s['local'])
total_public = sum(1 for s in refs_status.values() if s['public'])
print(f"Local: {total_local}/{len(REFS_PATHS)} arquivos")
print(f"Public: {total_public}/{len(REFS_PATHS)} arquivos")

In [ ]:
# Verificar estrutura de deploy - arquivos que devem ir para Vercel
print("\n🔍 VERIFICAÇÃO DE ESTRUTURA DE DEPLOY")
print("="*50)

# Verificar se os arquivos estão sendo incluídos no git
try:
    os.chdir(PROJECT_PATH)
    
    # Verificar git status
    git_status = subprocess.run(['git', 'status', '--porcelain'], 
                               capture_output=True, text=True)
    print(f"Git Status: {len(git_status.stdout.splitlines())} arquivos pendentes")
    
    # Verificar se refs estão no git
    git_ls = subprocess.run(['git', 'ls-files', 'refs/out/*.json'], 
                           capture_output=True, text=True)
    git_refs = git_ls.stdout.strip().split('\n') if git_ls.stdout.strip() else []
    print(f"Refs no Git: {len(git_refs)} arquivos")
    
    for ref in git_refs[:5]:  # Mostrar apenas primeiros 5
        print(f"   ✅ {ref}")
        
    # Verificar se public/refs estão no git
    git_public_ls = subprocess.run(['git', 'ls-files', 'public/refs/out/*.json'], 
                                  capture_output=True, text=True)
    git_public_refs = git_public_ls.stdout.strip().split('\n') if git_public_ls.stdout.strip() else []
    print(f"Public Refs no Git: {len(git_public_refs)} arquivos")
    
except Exception as e:
    print(f"❌ Erro ao verificar git: {e}")

# Verificar se existe .gitignore bloqueando refs
gitignore_path = os.path.join(PROJECT_PATH, ".gitignore")
if os.path.exists(gitignore_path):
    with open(gitignore_path, 'r') as f:
        gitignore_content = f.read()
        if 'refs/' in gitignore_content:
            print("⚠️ ATENÇÃO: .gitignore pode estar bloqueando arquivos refs/")
        else:
            print("✅ .gitignore não bloqueia arquivos refs/")

## 4. Teste de Deploy na Vercel

Executando deploy na Vercel com verificações de integridade.

In [ ]:
# Preparar e executar deploy na Vercel
print("🚀 TESTE DE DEPLOY NA VERCEL")
print("="*50)

# Função para verificar URLs na Vercel
def verificar_url_vercel(url):
    try:
        response = requests.get(url, timeout=10)
        return {
            "status": response.status_code,
            "exists": response.status_code == 200,
            "size": len(response.content) if response.status_code == 200 else 0
        }
    except Exception as e:
        return {
            "status": "error",
            "exists": False,
            "error": str(e)
        }

# Verificar se refs estão acessíveis na Vercel
print("🔍 Verificando arquivos de referência na Vercel...")

vercel_refs_check = {}
for ref_path in REFS_PATHS:
    # Testar diferentes caminhos possíveis
    test_urls = [
        f"{VERCEL_URL}/{ref_path}",
        f"{VERCEL_URL}/public/{ref_path}",
        f"{VERCEL_URL}/{ref_path.replace('refs/', '')}",
    ]
    
    print(f"\n📄 Testando {ref_path}:")
    
    for url in test_urls:
        result = verificar_url_vercel(url)
        status_icon = "✅" if result["exists"] else "❌"
        print(f"   {status_icon} {url} - Status: {result['status']}")
        
        if result["exists"]:
            vercel_refs_check[ref_path] = url
            break
    else:
        vercel_refs_check[ref_path] = None

print(f"\n📊 RESULTADO:")
found_refs = sum(1 for url in vercel_refs_check.values() if url is not None)
print(f"Refs encontradas na Vercel: {found_refs}/{len(REFS_PATHS)}")

In [ ]:
# Testar o scoring.js na Vercel
print("\n🧪 TESTANDO SCORING.JS NA VERCEL")
print("="*50)

# Verificar se o scoring.js está acessível
scoring_urls = [
    f"{VERCEL_URL}/lib/audio/features/scoring.js",
    f"{VERCEL_URL}/public/lib/audio/features/scoring.js"
]

for url in scoring_urls:
    result = verificar_url_vercel(url)
    status_icon = "✅" if result["exists"] else "❌"
    print(f"{status_icon} {url} - Status: {result['status']}")
    if result["exists"]:
        print(f"   Tamanho: {result['size']} bytes")

# Verificar página principal
main_page = verificar_url_vercel(VERCEL_URL)
print(f"\n🌐 Página principal: {'✅' if main_page['exists'] else '❌'} Status: {main_page['status']}")

# Se for necessário fazer novo deploy
print("\n🔄 PREPARANDO NOVO DEPLOY...")
print("1. Verificar se todos os arquivos estão commitados")
print("2. Executar git push para trigger deploy automático")
print("3. Aguardar deploy na Vercel")

# Executar git push se houver mudanças
try:
    os.chdir(PROJECT_PATH)
    
    # Verificar se há mudanças para commitar
    git_status = subprocess.run(['git', 'status', '--porcelain'], 
                               capture_output=True, text=True)
    
    if git_status.stdout.strip():
        print("📝 Há mudanças para commitar")
        print("💡 Execute: git add . && git commit -m 'fix: refs vercel' && git push")
    else:
        print("✅ Repositório limpo, pronto para deploy")
        
        # Fazer push para trigger deploy
        git_push = subprocess.run(['git', 'push', 'origin', 'main'], 
                                 capture_output=True, text=True)
        if git_push.returncode == 0:
            print("🚀 Push executado com sucesso - Deploy triggered")
        else:
            print(f"❌ Erro no push: {git_push.stderr}")
            
except Exception as e:
    print(f"❌ Erro ao executar git: {e}")

## 5. Validação de Funcionalidade

Testando a análise de áudio com diferentes gêneros para verificar scores.

In [ ]:
# Criar script de teste para validar funcionalidade na Vercel
print("🧪 CRIANDO SCRIPT DE VALIDAÇÃO")
print("="*50)

test_script = '''
// Script de teste para validar scoring na Vercel
async function testarScoringVercel() {
    console.log("🧪 TESTANDO SCORING NA VERCEL...");
    
    const generos = ["funk_mandela", "trance", "eletronico"];
    const resultados = {};
    
    for (const genero of generos) {
        try {
            // Dados de teste padrão
            const testData = {
                lufsIntegrated: -14.5,
                truePeakDbtp: -1.2,
                dr: 8.5,
                lra: 6.8,
                crestFactor: 9.2,
                stereoCorrelation: 0.25,
                stereoWidth: 0.65,
                balanceLR: 0.02,
                spectralCentroid: 2800,
                spectralFlatness: 0.22,
                spectralRolloff85: 9200,
                dcOffset: 0.01,
                clippingSamples: 0,
                clippingPct: 0
            };
            
            // Carregar scoring com cache bust
            const cacheBust = Date.now() + Math.random();
            const scoringModule = await import(`/lib/audio/features/scoring.js?v=${cacheBust}`);
            
            // Simular referência do gênero (fallback se não carregou)
            const mockRef = {
                lufs_target: genero === 'funk_mandela' ? -11.5 : -12.8,
                tol_lufs: 1.8,
                true_peak_target: -0.8,
                tol_true_peak: 1,
                dr_target: 7.2,
                tol_dr: 2,
                stereo_target: genero === 'funk_mandela' ? 0.38 : 0.42,
                tol_stereo: 0.15
            };
            
            const resultado = scoringModule.computeMixScore(testData, mockRef);
            
            resultados[genero] = {
                score: resultado?.score || resultado?.scorePct || 'N/A',
                method: resultado?.method || resultado?.scoringMethod || 'N/A',
                engine: resultado?.engineVersion || 'N/A',
                unified: resultado?.unifiedScoring || false
            };
            
            console.log(`✅ ${genero}: ${resultados[genero].score}%`);
            
        } catch (error) {
            console.error(`❌ Erro em ${genero}:`, error);
            resultados[genero] = { error: error.message };
        }
    }
    
    // Verificar se scores são diferentes
    const scores = Object.values(resultados)
        .filter(r => !r.error && r.score !== 'N/A')
        .map(r => r.score);
    
    const scoresDiferentes = new Set(scores).size > 1;
    
    console.log("📊 RESULTADO FINAL:");
    console.log("Scores diferentes:", scoresDiferentes ? "✅ SIM" : "❌ NÃO");
    console.log("Todos Engine 3.0.0:", Object.values(resultados).every(r => r.engine === '3.0.0') ? "✅ SIM" : "❌ NÃO");
    console.log("Todos unificados:", Object.values(resultados).every(r => r.unified === true) ? "✅ SIM" : "❌ NÃO");
    
    return resultados;
}

// Executar teste
testarScoringVercel();
'''

# Salvar script de teste
test_file_path = os.path.join(PROJECT_PATH, "teste-scoring-vercel-validation.js")
with open(test_file_path, 'w', encoding='utf-8') as f:
    f.write(test_script)

print(f"✅ Script de teste criado: {test_file_path}")
print("💡 Execute este script no console da Vercel para testar")

# Criar URL de teste direto
test_url = f"{VERCEL_URL}/teste-scoring-vercel.html"
print(f"🔗 URL de teste: {test_url}")

## 6. Correção de Bugs de Score

Implementando correções para garantir funcionamento adequado em produção.

In [ ]:
# Implementar correções críticas
print("🔧 IMPLEMENTANDO CORREÇÕES CRÍTICAS")
print("="*50)

# 1. Copiar refs para public se não existirem
print("1. Verificando e copiando arquivos de referência...")

import shutil

refs_copied = 0
for ref_path in REFS_PATHS:
    source = os.path.join(PROJECT_PATH, ref_path)
    target = os.path.join(PROJECT_PATH, "public", ref_path)
    
    if os.path.exists(source) and not os.path.exists(target):
        # Criar diretório se não existir
        os.makedirs(os.path.dirname(target), exist_ok=True)
        
        # Copiar arquivo
        shutil.copy2(source, target)
        print(f"   ✅ Copiado: {ref_path}")
        refs_copied += 1
    elif os.path.exists(target):
        print(f"   ℹ️ Já existe: {ref_path}")
    else:
        print(f"   ❌ Fonte não encontrada: {ref_path}")

print(f"📄 Arquivos copiados: {refs_copied}")

# 2. Criar fallback robusto para refs
print("\n2. Criando sistema de fallback para referências...")

fallback_refs = {
    "funk_mandela": {
        "lufs_target": -11.5,
        "tol_lufs": 1.8,
        "true_peak_target": -0.8,
        "tol_true_peak": 1,
        "dr_target": 7.2,
        "tol_dr": 2,
        "stereo_target": 0.38,
        "tol_stereo": 0.15
    },
    "trance": {
        "lufs_target": -12.8,
        "tol_lufs": 1.9,
        "true_peak_target": -0.8,
        "tol_true_peak": 1,
        "dr_target": 7.2,
        "tol_dr": 2,
        "stereo_target": 0.42,
        "tol_stereo": 0.14
    },
    "eletronico": {
        "lufs_target": -12.8,
        "tol_lufs": 1.9,
        "true_peak_target": -0.8,
        "tol_true_peak": 1,
        "dr_target": 7.2,
        "tol_dr": 2,
        "stereo_target": 0.42,
        "tol_stereo": 0.14
    }
}

# Salvar fallback em arquivo JS
fallback_js = f'''
// Fallback de referências para Vercel
window.FALLBACK_REFS = {json.dumps(fallback_refs, indent=2)};

// Função para obter referência com fallback
window.getRefWithFallback = function(genero) {{
    return window.FALLBACK_REFS[genero] || window.FALLBACK_REFS.funk_mandela;
}};

console.log("✅ Sistema de fallback de referências carregado");
'''

fallback_file = os.path.join(PROJECT_PATH, "public", "refs-fallback.js")
with open(fallback_file, 'w', encoding='utf-8') as f:
    f.write(fallback_js)

print(f"✅ Fallback criado: {fallback_file}")

In [ ]:
# 3. Verificar e corrigir o sistema de scoring
print("\n3. Verificando sistema de scoring...")

scoring_js_path = os.path.join(PROJECT_PATH, "lib", "audio", "features", "scoring.js")
public_scoring_path = os.path.join(PROJECT_PATH, "public", "lib", "audio", "features", "scoring.js")

if os.path.exists(scoring_js_path):
    print("✅ scoring.js encontrado na pasta lib")
    
    # Verificar se existe na pasta public
    if not os.path.exists(public_scoring_path):
        print("⚠️ scoring.js não existe na pasta public, copiando...")
        os.makedirs(os.path.dirname(public_scoring_path), exist_ok=True)
        shutil.copy2(scoring_js_path, public_scoring_path)
        print("✅ scoring.js copiado para public")
    else:
        print("✅ scoring.js já existe na pasta public")
        
    # Verificar conteúdo do scoring.js
    with open(scoring_js_path, 'r', encoding='utf-8') as f:
        content = f.read()
        
    if "3.0.0" in content and "equal_weight_v3" in content:
        print("✅ Scoring.js contém código unificado")
    else:
        print("❌ Scoring.js pode não ter código unificado")
        
else:
    print("❌ scoring.js não encontrado!")

# 4. Commitar todas as mudanças
print("\n4. Preparando commit com correções...")

try:
    os.chdir(PROJECT_PATH)
    
    # Add all changes
    subprocess.run(['git', 'add', '.'], check=True)
    
    # Commit
    commit_msg = "fix: correções para deploy vercel - refs fallback e scoring public"
    result = subprocess.run(['git', 'commit', '-m', commit_msg], 
                           capture_output=True, text=True)
    
    if result.returncode == 0:
        print("✅ Commit realizado com sucesso")
        
        # Push
        push_result = subprocess.run(['git', 'push', 'origin', 'main'], 
                                   capture_output=True, text=True)
        if push_result.returncode == 0:
            print("✅ Push realizado - Deploy triggered na Vercel")
        else:
            print(f"❌ Erro no push: {push_result.stderr}")
    else:
        if "nothing to commit" in result.stdout:
            print("ℹ️ Nenhuma mudança para commitar")
        else:
            print(f"❌ Erro no commit: {result.stderr}")
            
except Exception as e:
    print(f"❌ Erro ao executar git: {e}")

print("\n🎯 CORREÇÕES IMPLEMENTADAS:")
print("✅ Arquivos de referência copiados para public/")
print("✅ Sistema de fallback criado")
print("✅ scoring.js verificado e copiado")
print("✅ Deploy triggered na Vercel")
print("\n⏳ Aguarde 2-3 minutos para o deploy completar")

## Resumo Final e Próximos Passos

### ✅ Problemas Identificados e Corrigidos:

1. **404 em arquivos de referência**: Copiados para `public/refs/out/`
2. **Scoring.js não acessível**: Copiado para `public/lib/audio/features/`
3. **Sistema de fallback**: Implementado para casos de falha
4. **Deploy triggered**: Push realizado para Vercel

### 🧪 Validação Necessária:

1. **Aguardar deploy (2-3 minutos)**
2. **Testar no console da Vercel**: Execute o script `teste-scoring-vercel-validation.js`
3. **Verificar scores diferentes**: Cada gênero deve retornar score diferente
4. **Confirmar Engine 3.0.0**: Todos os resultados devem ter `engineVersion: "3.0.0"`

### 🔗 URLs de Teste:
- **Página principal**: https://ai-synth.vercel.app
- **Teste de scoring**: https://ai-synth.vercel.app/teste-scoring-vercel.html
- **Arquivo de referência**: https://ai-synth.vercel.app/public/refs/out/funk_mandela.json

### ⚡ Comandos de Emergência:
Se ainda houver problemas, execute no terminal local:
```bash
cd "c:\Users\DJ Correa\Desktop\Programação\ai-synth"
git add . && git commit -m "emergency fix vercel" && git push
```